## Final Project Submission

Please fill out:
* Student name: Paul Narducci
* Student pace: self paced 
* Scheduled project review date/time: 
* Instructor name: Jeff Herman
* Blog post URL:


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, make_scorer

In [10]:
# Load in CSV files and merge both dataframes into one dataframe. Droping id column because it holds no importance.

df_one = pd.read_csv('Data/Training_Set_Labels.csv')
df_one = df_one.drop('id', axis=1)

df_two =pd.read_csv('Data/Training_Set_Values.csv')
df_two = df_two.drop('id', axis=1)

tww_df = pd.concat([df_one['status_group'], df_two], axis=1)
tww_df.head(50)

,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322e+00,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329e+00,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-1.115530e+01,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359e+00,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
5,functional,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587e+00,Tajiri,0,...,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe
6,non functional,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365e+00,Kwa Ngomho,0,...,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
7,non functional,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198e+00,Tushirikiane,0,...,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
8,non functional,0.0,2012-11-03,Wateraid,0,Water Aid,32.711100,-5.146712e+00,Kwa Ramadhan Musa,0,...,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump
9,functional,0.0,2011-08-03,Isingiro Ho,0,Artisan,30.626991,-1.257051e+00,Kwapeto,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [11]:
# Let's look at the data and adjust datatypes or missing values where needed
tww_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   status_group           59400 non-null  object 
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [12]:
# We notice that our data is made up of objects, floats and ints. However, let's convert our date into datetime.
tww_df['date_recorded'] =  pd.to_datetime(tww_df['date_recorded'], format='%Y/%m/%d')

In [13]:
# This will allow us to look for '?' values in our dataset
for col in tww_df.columns:
    print(col, '\n', tww_df[col].value_counts(normalize=True).head(), '\n\n')

status_group 
 functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64 


amount_tsh 
 0.0       0.700993
500.0     0.052222
50.0      0.041616
1000.0    0.025051
20.0      0.024630
Name: amount_tsh, dtype: float64 


date_recorded 
 2011-03-15    0.009630
2011-03-17    0.009394
2013-02-03    0.009192
2011-03-14    0.008754
2011-03-16    0.008636
Name: date_recorded, dtype: float64 


funder 
 Government Of Tanzania    0.162898
Danida                    0.055841
Hesawa                    0.039487
Rwssp                     0.024639
World Bank                0.024191
Name: funder, dtype: float64 


gps_height 
  0     0.344074
-15    0.001010
-16    0.000926
-13    0.000926
-20    0.000875
Name: gps_height, dtype: float64 


installer 
 DWE           0.312171
Government    0.032738
RWE           0.021634
Commu         0.019015
DANIDA        0.018836
Name: installer, dtype: float64 


longitude 
 0.0000

There are no missing values that need to be converted to NaN values in this dataset. Let's look at our NaN values and figure out how best to handle these.

In [14]:
# Check for missing values
print(tww_df.isnull().sum())

status_group                 0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

We notice that funder, installer, subvillage, public_meeting, scheme_management, scheme_name, and permit are missing values. However, many of these features are similar and repetitive in what they are describing. First, I will drop the repetitive features and then fix the missing values for the remaining features.

In [15]:
# First, let's remove the features that are not needed in this data set due to redundancy with other features or they hold no significant value
tww_df = tww_df.drop(['date_recorded', 'num_private', 'subvillage', 'region_code', 'district_code', 'lga', 'ward', 'public_meeting', 'recorded_by', 'scheme_name', 'waterpoint_type', 'source', 'quantity_group', 'water_quality', 'payment', 'extraction_type_group', 'extraction_type', 'management', 'scheme_management', 'scheme_name', 'source_type'], axis=1)

In [16]:
# Let's also convert permit to a bool data type by replacing all NaN values with True
tww_df['permit'] = tww_df['permit'].fillna(tww_df['permit'].value_counts().idxmax())

#and then convert permit to an int
tww_df['permit'] = tww_df['permit'].astype(np.int64)

In [17]:
# Let's deal with our remaining NaN values for funder, installer, and scheme_manager
# We will use the mode in this case to replace our NaN values.
tww_df['funder'] = tww_df['funder'].fillna(tww_df['funder'].value_counts().idxmax())
tww_df['installer'] = tww_df['installer'].fillna(tww_df['installer'].value_counts().idxmax())

In [18]:
# In order to create consistency in our data, let's ensure that all the string data types are lowercase. 
tww_df['funder'] = tww_df['funder'].str.lower()
tww_df['installer'] = tww_df['installer'].str.lower()
tww_df['wpt_name'] = tww_df['wpt_name'].str.lower()
tww_df['basin'] = tww_df['basin'].str.lower()
tww_df['region'] = tww_df['region'].str.lower()

In [19]:
# Since some of the features have many categories, this will create issues when I run my model.
# The code below will check which categories account for less than 1% of the data and place them into a cetgory called other
series = pd.value_counts(tww_df['funder'])
mask = (series/series.sum() * 100).lt(1)
tww_df['funder'] = np.where(tww_df['funder'].isin(series[mask].index),'Other',tww_df['funder'])

series = pd.value_counts(tww_df['installer'])
mask = (series/series.sum() * 100).lt(1)
tww_df['installer'] = np.where(tww_df['installer'].isin(series[mask].index),'Other',tww_df['installer'])

series = pd.value_counts(tww_df['wpt_name'])
mask = (series/series.sum() * 100).lt(1)
tww_df['wpt_name'] = np.where(tww_df['wpt_name'].isin(series[mask].index),'Other',tww_df['wpt_name'])

series = pd.value_counts(tww_df['region'])
mask = (series/series.sum() * 100).lt(1)
tww_df['region'] = np.where(tww_df['region'].isin(series[mask].index),'Other',tww_df['region'])

In [20]:
# Create dummy variables for target and feature variables
tww_target = tww_df[['status_group']]
feat_dummies = tww_df[['funder', 'installer', 'wpt_name', 'basin', 'region', 'extraction_type_class', 'management_group', 'payment_type', 'quality_group', 'quantity', 'source_class', 'waterpoint_type_group']]

feat_dummies = pd.get_dummies(feat_dummies, drop_first=True)

#Create new feature variable with dummies
tww_features = pd.concat([feat_dummies, tww_df[['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'permit', 'construction_year']]], axis=1)
tww_features.head()

,funder_Other,funder_danida,funder_dhv,funder_district council,funder_dwsp,funder_germany republi,funder_government of tanzania,funder_hesawa,funder_kkkt,funder_norad,...,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other,amount_tsh,gps_height,longitude,latitude,population,permit,construction_year
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,6000.0,1390,34.938093,-9.856322,109,0,1999
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,1399,34.698766,-2.147466,280,1,2010
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,25.0,686,37.460664,-3.821329,250,1,2009
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,263,38.486161,-11.155298,58,1,1986
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,31.130847,-1.825359,0,1,0


## Run k-Nearest Neighbors

In [113]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
X_train, X_test, y_train, y_test = train_test_split(tww_features, tww_target, test_size=None, random_state=42)

In [111]:
from sklearn.preprocessing import MinMaxScaler
scaler_minmax = MinMaxScaler()

df_minmax_train = scaler_minmax.fit_transform(X_train)
df_minmax_test = scaler_minmax.transform(X_test)

In [132]:
grid_params = {
    'n_neighbors': [3,5,11,19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'],
}

gs = GridSearchCV(
     KNeighborsClassifier(),
     grid_params,
     verbose = 1,
     cv = 3,
     n_jobs = -1)

gs_results = gs.fit(df_minmax_train, y_train)

gs_results.best_estimator_

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 14.5min finished
C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


KNeighborsClassifier(metric='manhattan', n_neighbors=11, weights='distance')

In [35]:
#Run k-NN
knn_clf = KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance')

In [19]:
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average='weighted')))
    print("Recall Score: {}".format(recall_score(labels, preds, average='weighted')))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average='weighted')))

In [36]:
knn_clf.fit(df_minmax_train, y_train)

test_preds_two = knn_clf.predict(df_minmax_test)

print("Accuracy Score: {}".format(accuracy_score(y_test, test_preds_two)))

C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


Accuracy Score: 0.78


In [ ]:
roc_auc_score(y_test, test_preds_two)

In [ ]:
minmax_data_feat = scaler_minmax.fit_transform(tww_features)

cv_scores_knn_minmax = cross_val_score(knn_clf, minmax_data_feat, tww_target, cv = 10)
np.mean(cv_scores_knn_minmax)

## Test k-NN on Testing Model

In [22]:
# Test on Testing Model

tww_test_df = pd.read_csv('Data/Testing_Set_Values.csv')

tww_test_df = tww_test_df.drop(['id', 'date_recorded', 'num_private', 'subvillage', 'region_code', 'district_code', 'lga', 'ward', 'public_meeting', 'recorded_by', 'scheme_name', 'waterpoint_type', 'source', 'quantity_group', 'water_quality', 'payment', 'extraction_type_group', 'extraction_type', 'management', 'scheme_management', 'scheme_name', 'source_type'], axis=1)

print(tww_test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             14850 non-null  float64
 1   funder                 13981 non-null  object 
 2   gps_height             14850 non-null  int64  
 3   installer              13973 non-null  object 
 4   longitude              14850 non-null  float64
 5   latitude               14850 non-null  float64
 6   wpt_name               14850 non-null  object 
 7   basin                  14850 non-null  object 
 8   region                 14850 non-null  object 
 9   population             14850 non-null  int64  
 10  permit                 14113 non-null  object 
 11  construction_year      14850 non-null  int64  
 12  extraction_type_class  14850 non-null  object 
 13  management_group       14850 non-null  object 
 14  payment_type           14850 non-null  object 
 15  qu

In [23]:
# Let's also convert permit to a bool data type by replacing all NaN values with True
tww_test_df['permit'] = tww_test_df['permit'].fillna(tww_test_df['permit'].value_counts().idxmax())

#and then convert permit to an int
tww_test_df['permit'] = tww_test_df['permit'].astype(np.int64)

In [24]:
# Let's deal with our remaining NaN values for funder, installer, and scheme_manager
# We will use the mode in this case to replace our NaN values.
tww_test_df['funder'] = tww_test_df['funder'].fillna(tww_test_df['funder'].value_counts().idxmax())
tww_test_df['installer'] = tww_test_df['installer'].fillna(tww_test_df['installer'].value_counts().idxmax())

In [25]:
# In order to create consistency in our data, let's ensure that all the string data types are lowercase. 
tww_test_df['funder'] = tww_test_df['funder'].str.lower()
tww_test_df['installer'] = tww_test_df['installer'].str.lower()
tww_test_df['wpt_name'] = tww_test_df['wpt_name'].str.lower()
tww_test_df['basin'] = tww_test_df['basin'].str.lower()
tww_test_df['region'] = tww_test_df['region'].str.lower()

In [26]:
# Since some of the features have many categories, this will create issues when I run my model.
# The code below will check which categories account for less than 1% of the data and place them into a cetgory called other
series = pd.value_counts(tww_test_df['funder'])
mask = (series/series.sum() * 100).lt(1)
tww_test_df['funder'] = np.where(tww_test_df['funder'].isin(series[mask].index),'Other',tww_test_df['funder'])

series = pd.value_counts(tww_test_df['installer'])
mask = (series/series.sum() * 100).lt(1)
tww_test_df['installer'] = np.where(tww_test_df['installer'].isin(series[mask].index),'Other',tww_test_df['installer'])

series = pd.value_counts(tww_test_df['wpt_name'])
mask = (series/series.sum() * 100).lt(1)
tww_test_df['wpt_name'] = np.where(tww_test_df['wpt_name'].isin(series[mask].index),'Other',tww_test_df['wpt_name'])

series = pd.value_counts(tww_test_df['region'])
mask = (series/series.sum() * 100).lt(1)
tww_test_df['region'] = np.where(tww_test_df['region'].isin(series[mask].index),'Other',tww_test_df['region'])

In [27]:
feat_test_dummies = tww_test_df[['funder', 'installer', 'wpt_name', 'basin', 'region', 'extraction_type_class', 'management_group', 'payment_type', 'quality_group', 'quantity', 'source_class', 'waterpoint_type_group']]
feat_test_dummies = pd.get_dummies(feat_test_dummies, drop_first=True)

tww_test_features = pd.concat([feat_test_dummies, tww_test_df[['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'permit', 'construction_year']]], axis=1)

In [80]:
df_minmax_test_df = scaler_minmax.fit_transform(tww_test_features)

test_preds_df = knn_clf.predict(df_minmax_test_df)



KeyboardInterrupt: 

In [27]:
test_preds_df.tolist()

NameError: name 'test_preds_df' is not defined

In [ ]:
test_predictions_df = pd.DataFrame(data=test_preds_df)

In [ ]:
test_predictions_df = test_predictions_df.rename(columns={0: 'status_group'})
test_predictions_df.head()

In [34]:
test_labels_df = pd.read_csv('Testing_Y_Values/SubmissionFormat.csv')

In [35]:
test_labels_df = df_two = test_labels_df.drop('status_group', axis=1)

In [36]:
test_labels_df

,id
0,50785
1,51630
2,17168
3,45559
4,49871
...,...
14845,39307
14846,18990
14847,28749
14848,33492


In [41]:
test_predictions_df = pd.concat([test_labels_df, test_predictions_df], axis=1)
test_predictions_df.head()

,id,status_group
0,50785,non functional
1,51630,non functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [42]:
test_predictions_df.to_csv(r'Testing_Y_Values/SubmissionFormatThree.csv', index = False)

## Random Forest Classifier

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [ ]:
tree_clf = DecisionTreeClassifier() 

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6]
}

gs_tree = GridSearchCV(tree_clf, param_grid, cv=3)
gs_tree.fit(X_train, y_train.values.ravel())

gs_tree.best_params_

In [38]:
dtc_clf = DecisionTreeClassifier(criterion='gini', max_depth=None, min_samples_leaf=5, min_samples_split=5) 

In [39]:
dtc_clf.fit(X_train, y_train)

DecisionTreeClassifier(min_samples_leaf=5, min_samples_split=5)

In [40]:
dtc_pred = dtc_clf.predict(X_test)

In [41]:
print("Testing Accuracy for Decision Tree Classifier: {:.4}%".format(accuracy_score(y_test, dtc_pred) * 100))

Testing Accuracy for Decision Tree Classifier: 76.09%


In [293]:
bagged_tree =  BaggingClassifier(RandomForestClassifier(criterion='entropy', max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators = 75, max_features=20, min_weight_fraction_leaf=0, random_state=42, verbose=1))

bagged_tree.fit(X_train, y_train)

C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    6.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    6.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    6.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    6.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elap

BaggingClassifier(base_estimator=RandomForestClassifier(criterion='entropy',
                                                        max_features=20,
                                                        min_samples_split=5,
                                                        min_weight_fraction_leaf=0,
                                                        n_estimators=75,
                                                        random_state=42,
                                                        verbose=1))

In [294]:
bagged_tree.score(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

0.905746352413019

In [176]:
bag_pred = bagged_tree.predict(X_test)

In [182]:
print("Testing Accuracy for Bagging Tree Classifier: {:.4}%".format(accuracy_score(y_test, bag_pred) * 100))

Testing Accuracy for Bagging Tree Classifier: 75.72%


In [42]:
random_clf = RandomForestClassifier()

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6]
}

gs_tree = GridSearchCV(random_clf, param_grid, cv=3)
gs_tree.fit(X_train, y_train.values.ravel())

gs_tree.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5}

In [29]:
forest = RandomForestClassifier(criterion='entropy', max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators = 75, max_features=20, min_weight_fraction_leaf=0, random_state=42, verbose=1)
forest.fit(X_train, y_train)

C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    9.9s finished


RandomForestClassifier(criterion='entropy', max_features=20,
                       min_samples_split=5, min_weight_fraction_leaf=0,
                       n_estimators=75, random_state=42, verbose=1)

In [30]:
forest.score(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.8s finished


0.9575084175084175

In [31]:
forest_pred = forest.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.2s finished


In [32]:
print("Testing Accuracy for Random Forest Classifier: {:.4}%".format(accuracy_score(y_test, forest_pred) * 100))

Testing Accuracy for Random Forest Classifier: 80.53%


In [37]:
test_preds_df = forest.predict(tww_test_features)
test_predictions_df = pd.DataFrame(data=test_preds_df)
test_predictions_df = test_predictions_df.rename(columns={0: 'status_group'})
test_predictions_df = pd.concat([test_labels_df, test_predictions_df], axis=1)
test_predictions_df.to_csv(r'Testing_Y_Values/SubmissionFormatSix.csv', index = False)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.2s finished


In [33]:
tww_test_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Columns: 101 entries, funder_Other to construction_year
dtypes: float64(3), int64(4), uint8(94)
memory usage: 2.1 MB


In [34]:
tww_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Columns: 101 entries, funder_Other to construction_year
dtypes: float64(3), int64(4), uint8(94)
memory usage: 8.5 MB


In [171]:
test_predictions_df

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,non functional
14846,18990,functional
14847,28749,functional
14848,33492,functional


In [172]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [173]:
# Instantiate an AdaBoostClassifier
adaboost_clf = AdaBoostClassifier(random_state=42)

# Instantiate an GradientBoostingClassifier
gbt_clf = GradientBoostingClassifier(random_state=42)

In [174]:
adaboost_clf.fit(X_train, y_train)

C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


AdaBoostClassifier(random_state=42)

In [175]:
gbt_clf.fit(X_train, y_train)

C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GradientBoostingClassifier(random_state=42)

In [176]:
# AdaBoost model predictions
adaboost_train_preds = adaboost_clf.predict(X_train)
adaboost_test_preds = adaboost_clf.predict(X_test)

# GradientBoosting model predictions
gbt_clf_train_preds = gbt_clf.predict(X_train)
gbt_clf_test_preds = gbt_clf.predict(X_test)

In [180]:
def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds, average='weighted')
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))
    
print("Training Metrics")
display_acc_and_f1_score(y_train, adaboost_train_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_train, gbt_clf_train_preds, model_name='Gradient Boosted Trees')
print("")
print("Testing Metrics")
display_acc_and_f1_score(y_test, adaboost_test_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_test, gbt_clf_test_preds, model_name='Gradient Boosted Trees')

Training Metrics
Model: AdaBoost
Accuracy: 0.717598204264871
F1-Score: 0.6862758910697759

Model: Gradient Boosted Trees
Accuracy: 0.7533782267115601
F1-Score: 0.7311890986349314

Testing Metrics
Model: AdaBoost
Accuracy: 0.7174410774410774
F1-Score: 0.6862666385570775

Model: Gradient Boosted Trees
Accuracy: 0.7473400673400673
F1-Score: 0.7245010331573767


In [116]:
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [297]:
minmax_feats = scaler_minmax.fit_transform(tww_features)

In [298]:
X_train, X_test, y_train, y_test = train_test_split(minmax_feats, tww_target, test_size=None, random_state=42)

In [299]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 74.78%
Validation accuracy: 74.49%


In [120]:
clf = xgb.XGBClassifier()

param_grid = {
    "learning_rate": [0.1],
    'max_depth': [6],
    'min_child_weight': [10],
    'subsample': [ 0.7],
    'n_estimators': [5, 30, 100, 250],
}

gs_tree = GridSearchCV(clf, param_grid, cv=3)
gs_tree.fit(X_train, y_train)

gs_tree.best_params_

{'learning_rate': 0.1,
 'max_depth': 6,
 'min_child_weight': 10,
 'n_estimators': 250,
 'subsample': 0.7}

In [121]:
clf = xgb.XGBClassifier(learning_rate=0.1, max_depth=6, min_child_weight=10, n_estimators=260, max_features=20, subsample=0.7)
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 82.22%
Validation accuracy: 78.79%


In [39]:
import category_encoders as ce

def target_encode_multiclass(X,y): #X,y are pandas df and series
    y=y.astype(str)   #convert to string to onehot encode
    enc=ce.OneHotEncoder().fit(y)
    y_onehot=enc.transform(y)
    class_names=y_onehot.columns  #names of onehot encoded columns
    X_obj=X.select_dtypes('object') #separate categorical columns
    X=X.select_dtypes(exclude='object') 
    for class_ in class_names:
      
        enc=ce.TargetEncoder()
        enc.fit(X_obj,y_onehot[class_]) #convert all categorical 
        temp=enc.transform(X_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        X=pd.concat([X,temp],axis=1)    #add to original dataset
      
    return X

In [43]:
tww_y = tww_df[['status_group']]
tww_X = tww_df.drop('status_group', axis=1)

In [44]:
tww_y.head()

,status_group
0,functional
1,functional
2,functional
3,non functional
4,functional


In [45]:
tww_X.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,region,population,permit,construction_year,extraction_type_class,management_group,payment_type,quality_group,quantity,source_class,waterpoint_type_group
0,6000.0,Other,1390,Other,34.938093,-9.856322,none,lake nyasa,iringa,109,0,1999,gravity,user-group,annually,good,enough,groundwater,communal standpipe
1,0.0,Other,1399,Other,34.698766,-2.147466,zahanati,lake victoria,mara,280,1,2010,gravity,user-group,never pay,good,insufficient,surface,communal standpipe
2,25.0,Other,686,world vision,37.460664,-3.821329,Other,pangani,manyara,250,1,2009,gravity,user-group,per bucket,good,enough,surface,communal standpipe
3,0.0,unicef,263,Other,38.486161,-11.155298,Other,ruvuma / southern coast,mtwara,58,1,1986,submersible,user-group,never pay,good,dry,groundwater,communal standpipe
4,0.0,Other,0,Other,31.130847,-1.825359,shuleni,lake victoria,kagera,0,1,0,gravity,other,never pay,good,seasonal,surface,communal standpipe


In [46]:
target_encode_multiclass(tww_X, tww_y)

C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,amount_tsh,gps_height,longitude,latitude,population,permit,construction_year,funder_status_group_1,installer_status_group_1,wpt_name_status_group_1,...,wpt_name_status_group_3,basin_status_group_3,region_status_group_3,extraction_type_class_status_group_3,management_group_status_group_3,payment_type_status_group_3,quality_group_status_group_3,quantity_status_group_3,source_class_status_group_3,waterpoint_type_group_status_group_3
0,6000.0,1390,34.938093,-9.856322,109,0,1999,0.590815,0.571684,0.737728,...,0.021318,0.049164,0.023234,0.100859,0.074414,0.067820,0.076823,0.072320,0.059855,0.083986
1,0.0,1399,34.698766,-2.147466,280,1,2010,0.590815,0.571684,0.518072,...,0.093976,0.096507,0.030472,0.100859,0.074414,0.075233,0.076823,0.095842,0.117872,0.083986
2,25.0,686,37.460664,-3.821329,250,1,2009,0.590815,0.625179,0.532137,...,0.075483,0.053356,0.060644,0.100859,0.074414,0.045520,0.076823,0.072320,0.117872,0.083986
3,0.0,263,38.486161,-11.155298,58,1,1986,0.567644,0.571684,0.532137,...,0.075483,0.072557,0.072832,0.040945,0.074414,0.075233,0.076823,0.005924,0.059855,0.083986
4,0.0,0,31.130847,-1.825359,0,1,0,0.590815,0.571684,0.491419,...,0.081808,0.096507,0.091677,0.100859,0.059385,0.075233,0.076823,0.102716,0.117872,0.083986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,1210,37.169807,-3.253847,125,1,1999,0.881967,0.881967,0.532137,...,0.075483,0.053356,0.073533,0.100859,0.074414,0.045520,0.076823,0.072320,0.059855,0.083986
59396,4700.0,1212,35.249991,-9.070629,56,1,1996,0.590815,0.571684,0.532137,...,0.075483,0.054789,0.023234,0.100859,0.074414,0.067820,0.076823,0.072320,0.117872,0.083986
59397,0.0,0,34.017087,-8.750434,0,0,0,0.448227,0.542925,0.532137,...,0.075483,0.054789,0.108644,0.060464,0.074414,0.111687,0.059908,0.072320,0.059855,0.058840
59398,0.0,0,35.861315,-6.378573,0,1,0,0.590815,0.571684,0.532137,...,0.075483,0.054789,0.094957,0.060464,0.074414,0.075233,0.076823,0.095842,0.059855,0.058840


In [48]:
tww_X = target_encode_multiclass(tww_X, tww_y)

C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [49]:
tww_X

,amount_tsh,gps_height,longitude,latitude,population,permit,construction_year,funder_status_group_1,installer_status_group_1,wpt_name_status_group_1,...,wpt_name_status_group_3,basin_status_group_3,region_status_group_3,extraction_type_class_status_group_3,management_group_status_group_3,payment_type_status_group_3,quality_group_status_group_3,quantity_status_group_3,source_class_status_group_3,waterpoint_type_group_status_group_3
0,6000.0,1390,34.938093,-9.856322,109,0,1999,0.590815,0.571684,0.737728,...,0.021318,0.049164,0.023234,0.100859,0.074414,0.067820,0.076823,0.072320,0.059855,0.083986
1,0.0,1399,34.698766,-2.147466,280,1,2010,0.590815,0.571684,0.518072,...,0.093976,0.096507,0.030472,0.100859,0.074414,0.075233,0.076823,0.095842,0.117872,0.083986
2,25.0,686,37.460664,-3.821329,250,1,2009,0.590815,0.625179,0.532137,...,0.075483,0.053356,0.060644,0.100859,0.074414,0.045520,0.076823,0.072320,0.117872,0.083986
3,0.0,263,38.486161,-11.155298,58,1,1986,0.567644,0.571684,0.532137,...,0.075483,0.072557,0.072832,0.040945,0.074414,0.075233,0.076823,0.005924,0.059855,0.083986
4,0.0,0,31.130847,-1.825359,0,1,0,0.590815,0.571684,0.491419,...,0.081808,0.096507,0.091677,0.100859,0.059385,0.075233,0.076823,0.102716,0.117872,0.083986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,1210,37.169807,-3.253847,125,1,1999,0.881967,0.881967,0.532137,...,0.075483,0.053356,0.073533,0.100859,0.074414,0.045520,0.076823,0.072320,0.059855,0.083986
59396,4700.0,1212,35.249991,-9.070629,56,1,1996,0.590815,0.571684,0.532137,...,0.075483,0.054789,0.023234,0.100859,0.074414,0.067820,0.076823,0.072320,0.117872,0.083986
59397,0.0,0,34.017087,-8.750434,0,0,0,0.448227,0.542925,0.532137,...,0.075483,0.054789,0.108644,0.060464,0.074414,0.111687,0.059908,0.072320,0.059855,0.058840
59398,0.0,0,35.861315,-6.378573,0,1,0,0.590815,0.571684,0.532137,...,0.075483,0.054789,0.094957,0.060464,0.074414,0.075233,0.076823,0.095842,0.059855,0.058840


In [50]:
X_train, X_test, y_train, y_test = train_test_split(tww_X, tww_y, test_size=None, random_state=42)

In [51]:
random_clf = RandomForestClassifier()

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6]
}

gs_tree = GridSearchCV(random_clf, param_grid, cv=3)
gs_tree.fit(X_train, y_train.values.ravel())

gs_tree.best_params_

{'criterion': 'gini',
 'max_depth': None,
 'min_samples_leaf': 2,
 'min_samples_split': 5}

In [104]:
forest = RandomForestClassifier(criterion='gini', max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators = 75, max_features=20, random_state=42)
forest.fit(X_train, y_train)

C:\Users\paulv\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(max_features=20, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=75, random_state=42)

In [105]:
forest.score(X_train, y_train)

0.9358249158249158

In [106]:
forest_pred = forest.predict(X_test)

In [107]:
print("Testing Accuracy for Random Forest Classifier: {:.4}%".format(accuracy_score(y_test, forest_pred) * 100))

Testing Accuracy for Random Forest Classifier: 80.55%
